# Synchronous Hybrid Neuroevolution Notebook

This notebook implements the complete hybrid neuroevolution process synchronously, without the need for databases or external Kafka services. The system combines genetic algorithms with convolutional neural networks to evolve optimal architectures.

## Main Features:
- **Hybrid genetic algorithm**: Combines architecture and weight evolution
- **Synchronous processing**: Complete workflow executed in a single session
- **Configurable dataset**: Supports MNIST by default or custom dataset
- **Intelligent stopping criteria**: By target fitness or maximum generations
- **Complete visualization**: Shows progress and final best architecture

## Objectives:
1. Create initial population of CNN architectures
2. Evaluate fitness of each individual
3. Select best architectures (top 50%)
4. Apply crossover and mutation to create new generation
5. Repeat process until convergence
6. Display the best architecture found

## 1. Required Libraries Import

In [ ]:
# Install all necessary libraries
import subprocess
import sys

def install_package(package):
    """Installs a package using pip if not available."""
    try:
        __import__(package.split('==')[0].split('[')[0])
        print(f"OK {package.split('==')[0]} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"OK {package} installed correctly")

# List of required packages
required_packages = [
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "numpy>=1.21.0",
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "tqdm>=4.64.0",
    "jupyter>=1.0.0",
    "ipywidgets>=8.0.0"
]

print("Starting dependency installation for Hybrid Neuroevolution...")
print("=" * 60)

for package in required_packages:
    install_package(package)

print("\nAll dependencies have been verified/installed")
print("Restart the kernel if this is the first time installing torch")
print("=" * 60)

# Verify PyTorch installation
try:
    import torch
    print(f"\nPyTorch {torch.__version__} installed correctly")
    print(f"CUDA available: {'Yes' if torch.cuda.is_available() else 'No'}")
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
except ImportError:
    print("ERROR: PyTorch could not be installed correctly")
    print("Try installing manually with: pip install torch torchvision")

In [ ]:
# Main imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Scientific libraries
import numpy as np
import random
import copy
import json
import os
import threading
from typing import Dict, List, Tuple, Any
from datetime import datetime
import uuid

# Visualization and progress
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device configured: {device}")
print(f"PyTorch version: {torch.__version__}")

# Suppress unnecessary warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## 2. System Configuration and Parameters

In [ ]:
# Main genetic algorithm configuration (updated for adaptive mutation & moderate elitism)
CONFIG = {
    # Genetic algorithm parameters
    'population_size': 10,            # Population size
    'max_generations': 30,            # Maximum number of generations
    'fitness_threshold': 99.9,        # Target fitness (% accuracy)

    # Adaptive mutation parameters
    'base_mutation_rate': 0.35,       # Starting mutation rate (moderate)
    'mutation_rate_min': 0.10,        # Lower bound for adaptive mutation
    'mutation_rate_max': 0.80,        # Upper bound for adaptive mutation
    'current_mutation_rate': 0.35,    # Will be updated dynamically each generation

    'crossover_rate': 0.99,           # Crossover rate
    'elite_percentage': 0.2,          # Moderate elitism (20%) instead of 40%

    # Dataset selection
    'dataset': 'MNIST',             # Options: 'MNIST', 'CIFAR10', 'CUSTOM'

    # Dataset parameters (auto-configured based on dataset)
    'num_channels': 3,                # Input channels (1=grayscale, 3=RGB)
    'px_h': 32,                       # Image height
    'px_w': 32,                       # Image width
    'num_classes': 10,                # Number of classes
    'batch_size': 128,                # Batch size
    'test_split': 0.35,               # Validation percentage (for CUSTOM)

    # Training parameters
    'num_epochs': 12,                 # Max training epochs per evaluation (may stop earlier)
    'learning_rate': 0.01,            # Base learning rate (used only if genome doesn't override)
    'early_stopping_patience': 1000,  # Max batches per epoch (quick partial epoch)

    # Epoch-level early stopping (new)
    'epoch_patience': 3,              # Stop if no significant improvement after N evaluations
    'improvement_threshold': 0.2,     # Minimum (absolute) accuracy gain (%) to reset patience

    # Allowed architecture range
    'min_conv_layers': 1,
    'max_conv_layers': 7,
    'min_fc_layers': 1,
    'max_fc_layers': 7,
    'min_filters': 2,
    'max_filters': 256,
    'min_fc_nodes': 128,
    'max_fc_nodes': 2048,

    # Custom dataset configuration (only used if dataset='CUSTOM')
    'dataset_path': None,             # Custom dataset path
}

# Dataset configurations
DATASET_CONFIGS = {
    'MNIST': {
        'num_channels': 1,
        'px_h': 28,
        'px_w': 28,
        'num_classes': 10,
        'normalization': {'mean': (0.1307,), 'std': (0.3081,)}
    },
    'CIFAR10': {
        'num_channels': 3,
        'px_h': 32,
        'px_w': 32,
        'num_classes': 10,
        'normalization': {'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2023, 0.1994, 0.2010)}
    },
    'CUSTOM': {
        'num_channels': 1,  # Default, will be overridden
        'px_h': 28,         # Default, will be overridden
        'px_w': 28,         # Default, will be overridden
        'num_classes': 10,  # Default, will be overridden
        'normalization': {'mean': (0.5,), 'std': (0.5,)}
    }
}

# Auto-configure based on selected dataset
def configure_dataset(config, dataset_name):
    """Auto-configures dataset parameters based on selected dataset."""
    if dataset_name in DATASET_CONFIGS:
        dataset_config = DATASET_CONFIGS[dataset_name]
        config['num_channels'] = dataset_config['num_channels']
        config['px_h'] = dataset_config['px_h']
        config['px_w'] = dataset_config['px_w']
        config['num_classes'] = dataset_config['num_classes']
        config['_normalization'] = dataset_config['normalization']
    return config

# Configure the selected dataset
CONFIG = configure_dataset(CONFIG, CONFIG['dataset'])

# Activation function mapping
ACTIVATION_FUNCTIONS = {
    'relu': nn.ReLU,
    'leaky_relu': nn.LeakyReLU,
    'tanh': nn.Tanh,
    'sigmoid': nn.Sigmoid,
    'selu': nn.SELU,
}

# Optimizer mapping
OPTIMIZERS = {
    'adam': optim.Adam,
    'adamw': optim.AdamW,
    'sgd': optim.SGD,
    'rmsprop': optim.RMSprop,
}

print("Configuration loaded (adaptive mutation enabled):")
print(f"   Selected dataset: {CONFIG['dataset']}")
for key, value in CONFIG.items():
    if not key.startswith('_'):  # Hide internal config
        print(f"   {key}: {value}")
print(f"\nAvailable activation functions: {list(ACTIVATION_FUNCTIONS.keys())}")
print(f"Available optimizers: {list(OPTIMIZERS.keys())}")
print(f"Available datasets: {list(DATASET_CONFIGS.keys())}")

### Información sobre Datasets Disponibles

**MNIST**: 
- Dígitos escritos a mano (0-9)
- Imágenes en escala de grises (1 canal)
- Tamaño: 28x28 píxeles
- Dificultad: **Fácil** - Ideal para pruebas rápidas
- Fitness objetivo recomendado: >95%

**CIFAR-10**: 
- Objetos del mundo real (aviones, coches, pájaros, etc.)
- Imágenes en color (3 canales RGB)
- Tamaño: 32x32 píxeles
- Dificultad: **Media-Alta** - Más realista y desafiante
- Fitness objetivo recomendado: >80%
- Clases: plane, car, bird, cat, deer, dog, frog, horse, ship, truck

**CUSTOM**: 
- Tu propio dataset
- Configuración manual requerida
- Estructura de carpetas por clase

## 3. Dataset Loading and Preprocessing

In [ ]:
def load_dataset(config: dict) -> Tuple[DataLoader, DataLoader]:
    """
    Loads the dataset according to configuration.
    Returns train_loader and test_loader.
    """
    
    dataset_type = config['dataset']
    
    if dataset_type == 'CUSTOM' and config['dataset_path']:
        print(f"Loading custom dataset from: {config['dataset_path']}")
        
        # Transformations for custom dataset
        if config['num_channels'] == 1:
            normalize = transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        else:
            normalize = transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        
        transform = transforms.Compose([
            transforms.Resize((config['px_h'], config['px_w'])),
            transforms.ToTensor(),
            normalize
        ])
        
        # Load dataset from folders organized by class
        full_dataset = datasets.ImageFolder(root=config['dataset_path'], transform=transform)
        
        # Split into train and test
        train_size = int((1 - config['test_split']) * len(full_dataset))
        test_size = len(full_dataset) - train_size
        train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
        
        print(f"Custom dataset loaded:")
        print(f"   Classes found: {len(full_dataset.classes)}")
        print(f"   Total samples: {len(full_dataset)}")
        
    elif dataset_type == 'MNIST':
        print("Loading MNIST dataset...")
        
        # Transformations for MNIST
        transform = transforms.Compose([
            transforms.Resize((config['px_h'], config['px_w'])),
            transforms.ToTensor(),
            transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        ])
        
        # Load MNIST
        train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
        
        print(f"MNIST dataset loaded:")
        print(f"   Classes: {len(train_dataset.classes)}")
        print(f"   Training samples: {len(train_dataset)}")
        print(f"   Test samples: {len(test_dataset)}")
        
    elif dataset_type == 'CIFAR10':
        print("Loading CIFAR-10 dataset...")
        
        # Transformations for CIFAR-10
        transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),  # Data augmentation for training
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        ])
        
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(config['_normalization']['mean'], config['_normalization']['std'])
        ])
        
        # Load CIFAR-10
        train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
        test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
        
        print(f"CIFAR-10 dataset loaded:")
        print(f"   Classes: {train_dataset.classes}")
        print(f"   Training samples: {len(train_dataset)}")
        print(f"   Test samples: {len(test_dataset)}")
        
    else:
        raise ValueError(f"Dataset '{dataset_type}' not supported. Available: MNIST, CIFAR10, CUSTOM")
    
    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], 
        shuffle=True,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=config['batch_size'], 
        shuffle=False,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    return train_loader, test_loader

# Load the dataset
train_loader, test_loader = load_dataset(CONFIG)

# Get a sample to verify dimensions
sample_batch = next(iter(train_loader))
sample_data, sample_labels = sample_batch
print(f"\nDataset loaded successfully:")
print(f"   Batch shape: {sample_data.shape}")
print(f"   Data type: {sample_data.dtype}")
print(f"   Device: {sample_data.device}")
print(f"   Value range: [{sample_data.min():.3f}, {sample_data.max():.3f}]")

# Show some class information for CIFAR-10
if CONFIG['dataset'] == 'CIFAR10':
    cifar10_classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    print(f"   CIFAR-10 classes: {cifar10_classes}")
    unique_labels = torch.unique(sample_labels)
    print(f"   Labels in this batch: {unique_labels.tolist()}")

## 4. Neural Network Architecture Definition

In [ ]:
class EvolvableCNN(nn.Module):
    """
    Evolvable Convolutional Neural Network that builds architecture from genome specifications.
    
    This class implements a dynamically configurable CNN that constructs its architecture
    based on genetic parameters stored in a genome dictionary. It supports variable
    numbers of convolutional and fully connected layers with customizable parameters.
    
    Architecture Features:
    - Variable number of convolutional layers (1-7 layers)
    - Configurable filters, kernel sizes, and activation functions per layer
    - Batch normalization for training stability
    - Adaptive pooling based on layer position
    - Variable number of fully connected layers (1-7 layers)
    - Configurable dropout for regularization
    - Automatic output size calculation
    
    Genome Parameters Used:
    - num_conv_layers: Number of convolutional layers
    - num_fc_layers: Number of fully connected layers
    - filters: List of filter counts per conv layer
    - kernel_sizes: List of kernel sizes per conv layer
    - activations: List of activation function names
    - fc_nodes: List of neuron counts per FC layer
    - dropout_rate: Dropout probability for regularization
    
    Design Philosophy:
    - Flexible architecture that can represent diverse CNN designs
    - Automatic size calculation to handle variable input dimensions
    - Robust error handling for invalid genome configurations
    - Efficient forward pass implementation
    """
    
    def __init__(self, genome: dict, config: dict):
        """
        Initializes the evolvable CNN with genome and configuration parameters.
        
        Args:
            genome (dict): Genetic specification containing architecture parameters
            config (dict): System configuration with dataset and training parameters
        
        Construction Process:
        1. Store genome and configuration references
        2. Build convolutional layers based on genome specifications
        3. Calculate output dimensions after convolutions
        4. Build fully connected layers with proper input sizing
        
        Error Handling:
        - Validates genome parameters against configuration bounds
        - Provides meaningful error messages for invalid configurations
        - Ensures compatibility between layers
        """
        super(EvolvableCNN, self).__init__()
        self.genome = genome      # Genetic architecture specification
        self.config = config      # System configuration parameters
        
        # BUILD NETWORK ARCHITECTURE
        # Construct layers in dependency order
        
        # 1. Build convolutional feature extraction layers
        self.conv_layers = self._build_conv_layers()
        
        # 2. Calculate dimensions after convolutions for FC layer sizing
        self.conv_output_size = self._calculate_conv_output_size()
        
        # 3. Build fully connected classification layers
        self.fc_layers = self._build_fc_layers()
        
    def _build_conv_layers(self) -> nn.ModuleList:
        """
        Constructs convolutional layers according to genome specifications.
        
        This method builds the feature extraction part of the CNN with:
        - Variable number of convolutional layers
        - Batch normalization for training stability
        - Configurable activation functions
        - Adaptive pooling strategy
        
        Returns:
            nn.ModuleList: Sequential list of convolutional layer components
        
        Layer Structure (per convolutional layer):
        1. Conv2d: Feature extraction with specified filters and kernel size
        2. BatchNorm2d: Normalization for training stability
        3. Activation: Non-linear activation function from genome
        4. MaxPool2d: Spatial downsampling (adaptive based on layer position)
        
        Pooling Strategy:
        - Intermediate layers: MaxPool2d(2,2) for standard downsampling
        - Final layer: MaxPool2d(2,1) to preserve more spatial information
        """
        layers = nn.ModuleList()
        
        # Start with input channels from dataset configuration
        in_channels = self.config['num_channels']
        
        # BUILD EACH CONVOLUTIONAL LAYER
        for i in range(self.genome['num_conv_layers']):
            # Extract layer parameters from genome
            out_channels = self.genome['filters'][i]      # Number of filters
            kernel_size = self.genome['kernel_sizes'][i]  # Convolution kernel size
            
            # 1. CONVOLUTIONAL LAYER
            # Use padding=1 to maintain spatial dimensions initially
            conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
            layers.append(conv)
            
            # 2. BATCH NORMALIZATION
            # Normalizes activations for better training dynamics
            layers.append(nn.BatchNorm2d(out_channels))
            
            # 3. ACTIVATION FUNCTION
            # Apply activation specified in genome (with cycling for multiple layers)
            activation_name = self.genome['activations'][i % len(self.genome['activations'])]
            activation_func = ACTIVATION_FUNCTIONS[activation_name]()
            layers.append(activation_func)
            
            # 4. POOLING LAYER
            # Adaptive pooling strategy based on layer position
            if i < self.genome['num_conv_layers'] - 1:
                # Intermediate layers: standard 2x2 pooling for downsampling
                layers.append(nn.MaxPool2d(2, 2))
            else:
                # Final layer: preserve more spatial information with stride=1
                layers.append(nn.MaxPool2d(2, 1))
            
            # Update input channels for next layer
            in_channels = out_channels
            
        return layers
    
    def _calculate_conv_output_size(self) -> int:
        """Calculates output size after convolutional layers."""
        # Create dummy tensor to calculate size
        dummy_input = torch.zeros(1, self.config['num_channels'], 
                                 self.config['px_h'], self.config['px_w'])
        
        # Pass through convolutional layers
        x = dummy_input
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten and get size
        return x.view(-1).shape[0]
    
    def _build_fc_layers(self) -> nn.ModuleList:
        """Builds fully connected layers."""
        layers = nn.ModuleList()
        
        input_size = self.conv_output_size
        
        for i in range(self.genome['num_fc_layers']):
            output_size = self.genome['fc_nodes'][i]
            
            # Linear layer
            layers.append(nn.Linear(input_size, output_size))
            
            # Dropout if not last layer
            if i < self.genome['num_fc_layers'] - 1:
                layers.append(nn.Dropout(self.genome['dropout_rate']))
            
            input_size = output_size
        
        # Final classification layer
        layers.append(nn.Linear(input_size, self.config['num_classes']))
        
        return layers
    
    def forward(self, x):
        """Forward pass of the network."""
        # Convolutional layers
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        for i, layer in enumerate(self.fc_layers):
            x = layer(x)
            # Apply activation except on last layer
            if i < len(self.fc_layers) - 1 and not isinstance(layer, nn.Dropout):
                x = F.relu(x)
        
        return x
    
    def get_architecture_summary(self) -> str:
        """Returns an architecture summary."""
        summary = []
        summary.append(f"Conv Layers: {self.genome['num_conv_layers']}")
        summary.append(f"Filters: {self.genome['filters']}")
        summary.append(f"Kernel Sizes: {self.genome['kernel_sizes']}")
        summary.append(f"FC Layers: {self.genome['num_fc_layers']}")
        summary.append(f"FC Nodes: {self.genome['fc_nodes']}")
        summary.append(f"Activations: {self.genome['activations']}")
        summary.append(f"Dropout: {self.genome['dropout_rate']:.3f}")
        summary.append(f"Optimizer: {self.genome['optimizer']}")
        summary.append(f"Learning Rate: {self.genome['learning_rate']:.4f}")
        return " | ".join(summary)

print("EvolvableCNN class defined correctly")

## 5. Genetic Algorithm Components

In [ ]:
def create_random_genome(config: dict) -> dict:
    """Creates a random genome within specified ranges."""
    # Number of layers
    num_conv_layers = random.randint(config['min_conv_layers'], config['max_conv_layers'])
    num_fc_layers = random.randint(config['min_fc_layers'], config['max_fc_layers'])

    # Filters for each convolutional layer
    filters = [random.randint(config['min_filters'], config['max_filters']) for _ in range(num_conv_layers)]

    # Kernel sizes
    kernel_sizes = [random.choice([3, 5, 7]) for _ in range(num_conv_layers)]

    # Nodes in fully connected layers
    fc_nodes = [random.randint(config['min_fc_nodes'], config['max_fc_nodes']) for _ in range(num_fc_layers)]

    # Activation functions for each layer
    activations = [random.choice(list(ACTIVATION_FUNCTIONS.keys())) for _ in range(max(num_conv_layers, num_fc_layers))]

    # Other parameters
    dropout_rate = random.uniform(0.1, 0.5)
    learning_rate = random.choice([0.001, 0.0001, 0.01, 0.005])
    optimizer = random.choice(list(OPTIMIZERS.keys()))

    genome = {
        'num_conv_layers': num_conv_layers,
        'num_fc_layers': num_fc_layers,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'fc_nodes': fc_nodes,
        'activations': activations,
        'dropout_rate': dropout_rate,
        'learning_rate': learning_rate,
        'optimizer': optimizer,
        'fitness': 0.0,
        'id': str(uuid.uuid4())[:8]
    }
    return genome

def mutate_genome(genome: dict, config: dict) -> dict:
    """Applies mutation to a genome using adaptive mutation rate."""
    mutated_genome = copy.deepcopy(genome)
    mutation_rate = config['current_mutation_rate']  # adaptive

    # Mutate number of convolutional layers
    if random.random() < mutation_rate:
        mutated_genome['num_conv_layers'] = random.randint(config['min_conv_layers'], config['max_conv_layers'])
        num_conv = mutated_genome['num_conv_layers']
        mutated_genome['filters'] = mutated_genome['filters'][:num_conv]
        mutated_genome['kernel_sizes'] = mutated_genome['kernel_sizes'][:num_conv]
        while len(mutated_genome['filters']) < num_conv:
            mutated_genome['filters'].append(random.randint(config['min_filters'], config['max_filters']))
        while len(mutated_genome['kernel_sizes']) < num_conv:
            mutated_genome['kernel_sizes'].append(random.choice([1, 3, 5, 7]))

    # Mutate filters
    for i in range(len(mutated_genome['filters'])):
        if random.random() < mutation_rate:
            mutated_genome['filters'][i] = random.randint(config['min_filters'], config['max_filters'])

    # Mutate kernel sizes
    for i in range(len(mutated_genome['kernel_sizes'])):
        if random.random() < mutation_rate:
            mutated_genome['kernel_sizes'][i] = random.choice([1, 3, 5, 7])

    # Mutate number of FC layers
    if random.random() < mutation_rate:
        mutated_genome['num_fc_layers'] = random.randint(config['min_fc_layers'], config['max_fc_layers'])
        num_fc = mutated_genome['num_fc_layers']
        mutated_genome['fc_nodes'] = mutated_genome['fc_nodes'][:num_fc]
        while len(mutated_genome['fc_nodes']) < num_fc:
            mutated_genome['fc_nodes'].append(random.randint(config['min_fc_nodes'], config['max_fc_nodes']))

    # Mutate FC nodes
    for i in range(len(mutated_genome['fc_nodes'])):
        if random.random() < mutation_rate:
            mutated_genome['fc_nodes'][i] = random.randint(config['min_fc_nodes'], config['max_fc_nodes'])

    # Mutate activation functions
    for i in range(len(mutated_genome['activations'])):
        if random.random() < mutation_rate:
            mutated_genome['activations'][i] = random.choice(list(ACTIVATION_FUNCTIONS.keys()))

    # Mutate dropout
    if random.random() < mutation_rate:
        mutated_genome['dropout_rate'] = random.uniform(0.1, 0.8)

    # Mutate learning rate
    if random.random() < mutation_rate:
        mutated_genome['learning_rate'] = random.choice([0.001, 0.0001, 0.01, 0.005, 0.000001, 0.05, 0.00005, 0.0005])

    # Mutate optimizer
    if random.random() < mutation_rate:
        mutated_genome['optimizer'] = random.choice(list(OPTIMIZERS.keys()))

    mutated_genome['id'] = str(uuid.uuid4())[:8]
    mutated_genome['fitness'] = 0.0
    return mutated_genome

def crossover_genomes(parent1: dict, parent2: dict, config: dict) -> Tuple[dict, dict]:
    """Performs crossover between two genomes."""
    if random.random() > config['crossover_rate']:
        return copy.deepcopy(parent1), copy.deepcopy(parent2)

    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)

    # Crossover scalar parameters
    for key in ['num_conv_layers', 'num_fc_layers', 'dropout_rate', 'learning_rate', 'optimizer']:
        if random.random() < 0.5:
            child1[key], child2[key] = child2[key], child1[key]

    # Crossover lists (random cut point)
    for list_key in ['filters', 'kernel_sizes', 'fc_nodes', 'activations']:
        if random.random() < 0.5:
            list1 = child1[list_key]
            list2 = child2[list_key]
            if len(list1) > 1 and len(list2) > 1:
                point1 = random.randint(1, len(list1) - 1)
                point2 = random.randint(1, len(list2) - 1)
                child1[list_key] = list1[:point1] + list2[point2:]
                child2[list_key] = list2[:point2] + list1[point1:]

    child1['id'] = str(uuid.uuid4())[:8]
    child2['id'] = str(uuid.uuid4())[:8]
    child1['fitness'] = 0.0
    child2['fitness'] = 0.0
    return child1, child2

print("Genetic functions updated for adaptive mutation")

## 6. Hybrid Neuroevolution Implementation

In [ ]:
class HybridNeuroevolution:
    """
    Main class implementing hybrid neuroevolution with concurrent processing capabilities.
    
    This class combines genetic algorithms with neural network training to evolve optimal
    CNN architectures and hyperparameters. It includes advanced features for performance
    and reliability:
    
    Key Features:
    - Concurrent genome evaluation using threading (2x performance improvement)
    - Adaptive mutation rates based on population diversity
    - Interleaved training/evaluation with early stopping
    - Comprehensive convergence criteria and stagnation detection
    - Thread-safe operations with proper synchronization
    - Real-time progress monitoring and logging
    
    Threading Architecture:
    - Main thread: Coordinates evolution process and manages shared state
    - Worker threads: Process genomes concurrently for fitness evaluation
    - Synchronization: Uses locks to ensure thread-safe access to shared data
    
    Evolution Strategy:
    - Elitism: Preserves best individuals across generations
    - Tournament selection: Promotes diversity while favoring fitness
    - Adaptive operators: Adjusts mutation rates based on population state
    - Early stopping: Prevents overfitting and reduces computation time
    
    Attributes:
        config (dict): Configuration parameters for evolution and training
        train_loader (DataLoader): Training data loader
        test_loader (DataLoader): Test data loader for fitness evaluation
        population (list): Current population of genomes
        generation (int): Current generation number
        best_individual (dict): Best genome found across all generations
        fitness_history (list): History of best fitness per generation
        generation_stats (list): Detailed statistics for each generation
        
    Thread-Safe Attributes (used during evaluation):
        pending_genomes (list): Genomes waiting for evaluation
        completed_genomes (list): Genomes that have been evaluated
        fitness_scores (list): Fitness values for statistical analysis
        evaluation_log (list): Chronological log of thread operations
        genome_lock (Lock): Synchronizes access to genome lists
        results_lock (Lock): Synchronizes logging operations
    """

    def __init__(self, config: dict, train_loader: DataLoader, test_loader: DataLoader):
        """
        Initializes the HybridNeuroevolution system with configuration and data loaders.
        
        Args:
            config (dict): Configuration dictionary containing all evolution parameters
            train_loader (DataLoader): PyTorch DataLoader for training data
            test_loader (DataLoader): PyTorch DataLoader for test/validation data
        
        Initialization:
        - Stores configuration and data loaders for the evolution process
        - Initializes evolution state variables to default values
        - Prepares data structures for population management and statistics
        
        State Variables:
        - population: Will store the current generation of genomes
        - generation: Tracks the current generation number (starts at 0)
        - best_individual: Will store the best genome found across all generations
        - fitness_history: Tracks best fitness value per generation for convergence analysis
        - generation_stats: Stores detailed statistics for visualization and analysis
        """
        # CORE CONFIGURATION
        self.config = config  # Evolution parameters and hyperparameters
        self.train_loader = train_loader  # Training data for fitness evaluation
        self.test_loader = test_loader    # Test data for fitness evaluation
        
        # EVOLUTION STATE
        self.population = []  # Current population of genomes (initially empty)
        self.generation = 0   # Current generation counter (starts at 0)
        self.best_individual = None  # Best genome found so far (initially None)
        
        # STATISTICS AND MONITORING
        self.fitness_history = []     # Best fitness per generation for trend analysis
        self.generation_stats = []    # Detailed statistics per generation for visualization

    def initialize_population(self):
        """
        Creates the initial population of random genomes for evolution.
        
        This function generates a diverse set of random neural network architectures
        to serve as the starting point for the evolutionary process.
        
        Each genome contains:
        - Architecture parameters (number of layers, filters, nodes)
        - Hyperparameters (learning rate, optimizer, dropout)
        - Unique identifier for tracking
        
        Population Diversity:
        - Random architecture combinations within specified bounds
        - Random hyperparameter selection from predefined ranges
        - Ensures genetic diversity for effective evolution
        """
        print(f"Initializing population of {self.config['population_size']} individuals...")
        
        # CREATE RANDOM GENOMES
        # Generate population_size random genomes using the create_random_genome function
        # Each genome represents a complete neural network specification
        self.population = [create_random_genome(self.config) for _ in range(self.config['population_size'])]
        
        print(f"Population initialized with {len(self.population)} individuals")

    def _train_one_epoch(self, model, optimizer, criterion, genome_id: str, epoch: int):
        """
        Trains the model for one epoch with batch limiting for faster evaluation.
        
        This function implements a single training epoch with optimizations for
        neuroevolution where we need fast evaluation rather than perfect training.
        
        Args:
            model: PyTorch neural network model
            optimizer: Optimizer instance (Adam, SGD, etc.)
            criterion: Loss function (CrossEntropyLoss)
            genome_id: Unique identifier for logging
            epoch: Current epoch number
        
        Returns:
            float: Average loss for this epoch
        
        Optimizations:
        - Batch limiting: Process only a subset of batches for speed
        - Early exit: Stop after reaching batch limit
        - Memory efficient: Uses standard training loop without unnecessary operations
        """
        model.train()  # Set model to training mode (enables dropout, batch norm updates)
        running_loss = 0.0  # Accumulator for loss values
        batch_count = 0  # Counter for processed batches
        
        # BATCH LIMITING FOR SPEED
        # Process only a limited number of batches per epoch to speed up evaluation
        # This is acceptable in neuroevolution where we need relative fitness comparison
        max_batches = min(len(self.train_loader), self.config['early_stopping_patience'])
        
        # TRAINING LOOP
        for data, target in self.train_loader:
            # Move data to appropriate device (GPU/CPU)
            data, target = data.to(device), target.to(device)
            
            # STANDARD PYTORCH TRAINING STEP
            optimizer.zero_grad()  # Clear gradients from previous iteration
            output = model(data)   # Forward pass
            loss = criterion(output, target)  # Calculate loss
            loss.backward()        # Backward pass (calculate gradients)
            optimizer.step()       # Update model parameters
            
            # ACCUMULATE STATISTICS
            running_loss += loss.item()
            batch_count += 1
            
            # EARLY EXIT FOR SPEED
            if batch_count >= max_batches:
                break
        
        # CALCULATE AND REPORT AVERAGE LOSS
        avg_loss = running_loss / max(1, batch_count)
        print(f"          Train Epoch {epoch}: loss={avg_loss:.4f} ({batch_count} batches)")
        return avg_loss

    def _evaluate(self, model, criterion, genome_id: str, epoch: int):
        """
        Evaluates the model on test data to measure current performance.
        
        This function performs model evaluation with optimizations for neuroevolution
        where we need fast but reliable accuracy measurements.
        
        Args:
            model: PyTorch neural network model
            criterion: Loss function for evaluation
            genome_id: Unique identifier for logging
            epoch: Current epoch number
        
        Returns:
            tuple: (accuracy_percentage, average_loss)
        
        Optimizations:
        - Batch limiting: Evaluate on subset of test data for speed
        - No gradient computation: Uses torch.no_grad() for memory efficiency
        - Early exit: Stop after reaching evaluation batch limit
        """
        model.eval()  # Set model to evaluation mode (disables dropout, batch norm training)
        
        # EVALUATION STATISTICS
        correct = 0  # Count of correctly classified samples
        total = 0    # Total number of samples processed
        eval_batches = 0  # Number of evaluation batches processed
        total_eval_loss = 0.0  # Accumulator for evaluation loss
        
        # BATCH LIMITING FOR SPEED
        # Evaluate on limited number of batches to speed up the process
        # 20 batches usually provide reliable accuracy estimate
        max_eval_batches = min(len(self.test_loader), 20)
        
        # EVALUATION LOOP (NO GRADIENT COMPUTATION)
        with torch.no_grad():  # Disable gradient computation for memory efficiency
            for data, target in self.test_loader:
                # Move data to appropriate device (GPU/CPU)
                data, target = data.to(device), target.to(device)
                
                # FORWARD PASS ONLY
                output = model(data)  # Get model predictions
                loss = criterion(output, target)  # Calculate loss for monitoring
                total_eval_loss += loss.item()
                
                # ACCURACY CALCULATION
                _, predicted = torch.max(output, 1)  # Get class with highest probability
                total += target.size(0)  # Add batch size to total
                correct += (predicted == target).sum().item()  # Count correct predictions
                
                eval_batches += 1
                
                # EARLY EXIT FOR SPEED
                if eval_batches >= max_eval_batches:
                    break
        
        # CALCULATE FINAL METRICS
        accuracy = 100.0 * correct / max(1, total)  # Convert to percentage
        avg_eval_loss = total_eval_loss / max(1, eval_batches)  # Average loss
        
        print(f"          Eval  Epoch {epoch}: acc={accuracy:.2f}% loss={avg_eval_loss:.4f} ({eval_batches} batches)")
        return accuracy, avg_eval_loss

    def evaluate_fitness(self, genome: dict) -> float:
        """
        Evaluates a single genome's fitness by training and testing a neural network.
        
        This is the core fitness evaluation function that:
        1. Creates a CNN model from the genome specification
        2. Trains the model using interleaved train/eval epochs
        3. Implements early stopping based on improvement thresholds
        4. Returns the best accuracy achieved as fitness score
        
        Args:
            genome (dict): Genome dictionary containing architecture and hyperparameters
        
        Returns:
            float: Fitness score (accuracy percentage, 0.0-100.0)
        
        Training Strategy:
        - Interleaved training: alternates between training and evaluation each epoch
        - Early stopping: stops training if no significant improvement is detected
        - Batch limiting: limits batches per epoch for faster evaluation
        
        Error Handling:
        - Returns 0.0 fitness if any errors occur during training
        - Logs errors for debugging purposes
        """
        try:
            # MODEL CREATION
            # Instantiate CNN model from genome specification and move to GPU/CPU
            model = EvolvableCNN(genome, self.config).to(device)
            
            # OPTIMIZER SETUP
            # Create optimizer instance based on genome's optimizer choice
            optimizer_class = OPTIMIZERS[genome['optimizer']]
            optimizer = optimizer_class(model.parameters(), lr=genome['learning_rate'])
            
            # LOSS FUNCTION
            # Use CrossEntropyLoss for classification tasks
            criterion = nn.CrossEntropyLoss()

            # TRAINING STATE VARIABLES
            best_acc = 0.0  # Best accuracy achieved during training
            best_epoch = -1  # Epoch where best accuracy was achieved
            patience_left = self.config['epoch_patience']  # Early stopping patience counter
            last_improvement_acc = 0.0  # Last accuracy that triggered patience reset

            max_epochs = self.config['num_epochs']  # Maximum training epochs
            print(f"      Training/Evaluating model {genome['id']} (max {max_epochs} epochs interleaved)")

            # INTERLEAVED TRAINING/EVALUATION LOOP
            # Train for one epoch, then immediately evaluate - repeat
            for epoch in range(1, max_epochs + 1):
                # TRAINING PHASE
                # Train model for one epoch with batch limiting for speed
                self._train_one_epoch(model, optimizer, criterion, genome['id'], epoch)
                
                # EVALUATION PHASE
                # Immediately evaluate after training to get current performance
                acc, eval_loss = self._evaluate(model, criterion, genome['id'], epoch)

                # EARLY STOPPING LOGIC
                # Check if accuracy improved significantly enough to continue training
                improvement = acc - last_improvement_acc
                if improvement >= self.config['improvement_threshold']:
                    patience_left = self.config['epoch_patience']  # Reset patience
                    last_improvement_acc = acc  # Update improvement baseline
                else:
                    patience_left -= 1  # Decrease patience

                # BEST ACCURACY TRACKING
                if acc > best_acc:
                    best_acc = acc
                    best_epoch = epoch

                # PROGRESS REPORTING
                print(f"              -> Acc={acc:.2f}% (best={best_acc:.2f}% at epoch {best_epoch}) patience_left={patience_left}")

                # EARLY STOPPING CHECK
                if patience_left <= 0:
                    print(f"              Early stopping triggered (no significant improvement)")
                    break

            # FINAL RESULT
            print(f"      Final fitness for {genome['id']}: {best_acc:.2f}% (best epoch {best_epoch})")
            return best_acc
            
        except Exception as e:
            # ERROR HANDLING
            # Return 0.0 fitness and log error for debugging
            print(f"      ERROR evaluating genome {genome['id']}: {e}")
            logger.warning(f"Error evaluating genome {genome['id']}: {e}")
            return 0.0

    def _thread_worker(self, thread_name: str):
        """
        Thread worker function that processes genomes from the shared queue.
        
        This function implements the core threading logic for concurrent genome evaluation.
        Each thread continuously extracts genomes from a shared pending list, evaluates
        their fitness, and stores results in a shared completed list.
        
        Args:
            thread_name (str): Unique identifier for this thread (e.g., "THREAD-1")
        
        Thread Safety:
            - Uses self.genome_lock to synchronize access to shared genome lists
            - Uses self.results_lock to synchronize logging operations
            - Implements atomic operations to prevent race conditions
        
        Workflow:
            1. Extract genome from pending_genomes list (thread-safe)
            2. Evaluate genome fitness (time-consuming, outside locks)
            3. Store results in completed_genomes list (thread-safe)
            4. Repeat until no more genomes to process
        """
        operations_count = 0  # Counter for genomes processed by this thread
        
        print(f"      🚀 {thread_name} started")
        
        while True:
            # Initialize variables for current operation
            current_genome = None
            remaining_count = 0
            completed_count = 0
            
            # CRITICAL SECTION: Extract genome from pending list
            # This section must be atomic to prevent multiple threads from
            # extracting the same genome or corrupting the list state
            with self.genome_lock:
                if self.pending_genomes:  # Check if there are genomes to process
                    current_genome = self.pending_genomes.pop(0)  # Extract first genome (FIFO)
                    remaining_count = len(self.pending_genomes)  # Capture current state
                    completed_count = len(self.completed_genomes)  # Capture current state
                    operations_count += 1  # Increment this thread's counter
                else:
                    # No more genomes to process - thread should terminate
                    break
            
            # GENOME PROCESSING: Execute outside of locks to maximize concurrency
            # The actual fitness evaluation is the most time-consuming part and
            # should not be done inside a lock to allow other threads to work
            if current_genome:
                # Generate timestamp for logging (high precision for ordering)
                timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                
                # Log genome extraction (thread-safe logging)
                with self.results_lock:
                    self.evaluation_log.append({
                        'thread': thread_name,
                        'operation': f'EXTRACTED {current_genome["id"]}',
                        'pending_remaining': remaining_count,
                        'completed_count': completed_count,
                        'timestamp': timestamp
                    })
                
                # Display processing information (user feedback)
                print(f"\n      [{timestamp}] {thread_name} - Processing genome {current_genome['id']}")
                print(f"         Architecture: {current_genome['num_conv_layers']} conv + {current_genome['num_fc_layers']} fc, opt={current_genome['optimizer']}, lr={current_genome['learning_rate']}")
                print(f"         Remaining: {remaining_count} | Completed: {completed_count + 1}")
                
                # CORE OPERATION: Evaluate fitness (computationally intensive)
                # This is where the actual neural network training and evaluation occurs
                # Takes significant time (seconds to minutes per genome)
                fitness = self.evaluate_fitness(current_genome)
                current_genome['fitness'] = fitness  # Store fitness in genome
                
                # CRITICAL SECTION: Store results safely
                # Multiple threads may try to update shared state simultaneously
                with self.genome_lock:
                    self.completed_genomes.append(current_genome)  # Add to completed list
                    self.fitness_scores.append(fitness)  # Add to fitness tracking
                    
                    # Update global best fitness if this genome achieved a new record
                    if fitness > self.best_fitness_so_far:
                        self.best_fitness_so_far = fitness
                        print(f"         🎯 New best fitness in this generation: {fitness:.2f}%!")
                
                # Log completion (thread-safe logging)
                completion_timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                with self.results_lock:
                    self.evaluation_log.append({
                        'thread': thread_name,
                        'operation': f'COMPLETED {current_genome["id"]} - {fitness:.2f}%',
                        'pending_remaining': len(self.pending_genomes),
                        'completed_count': len(self.completed_genomes),
                        'timestamp': completion_timestamp
                    })
                
                # Display completion information
                print(f"         ✅ Fitness: {fitness:.2f}% | Best so far: {self.best_fitness_so_far:.2f}%")
        
        # Thread termination message
        print(f"      🏁 {thread_name} finished - {operations_count} genomes processed")

    def evaluate_population(self):
        """
        Evaluates the entire population using concurrent threading for improved performance.
        
        This method implements a producer-consumer pattern where:
        - The main thread acts as a coordinator, setting up shared data structures
        - Two worker threads act as consumers, processing genomes from a shared queue
        - All threads synchronize using locks to ensure thread safety
        
        Threading Strategy:
        - Uses 2 concurrent threads to process genomes in parallel (2x speedup)
        - Implements thread-safe access to shared data structures
        - Provides real-time logging and progress tracking
        
        Data Structures:
        - pending_genomes: Source list of genomes to be evaluated
        - completed_genomes: Destination list of evaluated genomes
        - fitness_scores: List of fitness values for statistical analysis
        - evaluation_log: Chronological log of thread operations
        
        Synchronization:
        - genome_lock: Protects access to genome lists and fitness tracking
        - results_lock: Protects logging operations to prevent garbled output
        """
        print(f"\nEvaluating population (Generation {self.generation})...")
        print(f"Processing {len(self.population)} individuals using 2 concurrent threads...")
        
        # SHARED DATA STRUCTURES INITIALIZATION
        # These structures are shared between threads and require synchronization
        self.pending_genomes = self.population.copy()  # Source: genomes waiting for evaluation
        self.completed_genomes = []  # Destination: genomes that have been evaluated
        self.fitness_scores = []  # Fitness values for statistical analysis
        self.best_fitness_so_far = 0.0  # Track best fitness in current generation
        
        # THREAD SYNCHRONIZATION MECHANISMS
        # These locks ensure thread-safe access to shared data structures
        self.genome_lock = threading.Lock()  # Protects genome lists and fitness data
        self.results_lock = threading.Lock()  # Protects logging and output operations
        self.evaluation_log = []  # Chronological log of all thread operations
        
        print(f"Starting threaded evaluation with {len(self.pending_genomes)} genomes...")
        
        # THREAD CREATION AND CONFIGURATION
        # Create two worker threads for concurrent genome processing
        thread1 = threading.Thread(target=self._thread_worker, args=("THREAD-1",))
        thread2 = threading.Thread(target=self._thread_worker, args=("THREAD-2",))
        
        # Configure threads as daemon threads for clean shutdown
        # Daemon threads automatically terminate when the main program exits
        thread1.daemon = True
        thread2.daemon = True
        
        # THREAD EXECUTION
        # Start both threads simultaneously for maximum concurrency
        thread1.start()
        thread2.start()
        
        print(f"   ✅ Threads started. Active threads: {threading.active_count()}")
        
        # THREAD SYNCHRONIZATION
        # Wait for both threads to complete their work before proceeding
        # This ensures all genomes have been processed before continuing
        thread1.join()
        thread2.join()
        
        # COMPLETION REPORTING
        print(f"\n💯 Threaded evaluation completed!")
        print(f"   Total genomes processed: {len(self.completed_genomes)}")
        print(f"   Best fitness found: {self.best_fitness_so_far:.2f}%")
        print(f"   Active threads: {threading.active_count()}")
        
        # OPERATION LOG DISPLAY
        # Show chronological sequence of operations for debugging and analysis
        print(f"\n--- Chronological Thread Operations Log ---")
        self.evaluation_log.sort(key=lambda x: x['timestamp'])  # Sort by timestamp
        for entry in self.evaluation_log:
            print(f"[{entry['timestamp']}] {entry['thread']} - {entry['operation']}")
        
        # DATA CONSOLIDATION
        # Update population with evaluated results and prepare for next generation
        self.population = self.completed_genomes.copy()  # Replace population with evaluated genomes
        fitness_scores = self.fitness_scores.copy()  # Get fitness scores for statistics
        
        # STATISTICAL ANALYSIS
        # Calculate generation statistics for monitoring evolution progress
        if fitness_scores:
            avg_fitness = np.mean(fitness_scores)  # Population average fitness
            max_fitness = np.max(fitness_scores)   # Best fitness in generation
            min_fitness = np.min(fitness_scores)   # Worst fitness in generation
            std_fitness = np.std(fitness_scores)   # Population diversity measure
        else:
            # Handle edge case where no valid fitness scores exist
            avg_fitness = max_fitness = min_fitness = std_fitness = 0.0

        # Store generation statistics for analysis and visualization
        stats = {
            'generation': self.generation,
            'avg_fitness': avg_fitness,
            'max_fitness': max_fitness,
            'min_fitness': min_fitness,
            'std_fitness': std_fitness
        }
        self.generation_stats.append(stats)
        self.fitness_history.append(max_fitness)  # Track fitness evolution over time

        # GLOBAL BEST TRACKING
        # Update global best individual if a new record was achieved
        best_genome = max(self.population, key=lambda x: x['fitness'])
        if self.best_individual is None or best_genome['fitness'] > self.best_individual['fitness']:
            self.best_individual = copy.deepcopy(best_genome)  # Deep copy to preserve state
            print(f"\nNew global best individual found!")

        # GENERATION SUMMARY REPORTING
        # Display comprehensive statistics for this generation
        print(f"\nGENERATION {self.generation} STATISTICS:")
        print(f"   Maximum fitness: {max_fitness:.2f}%")
        print(f"   Average fitness: {avg_fitness:.2f}%")
        print(f"   Minimum fitness: {min_fitness:.2f}%")
        print(f"   Standard deviation: {std_fitness:.2f}%")
        print(f"   Best individual: {best_genome['id']} with {best_genome['fitness']:.2f}%")
        print(f"   Global best individual: {self.best_individual['id']} with {self.best_individual['fitness']:.2f}%")

    def selection_and_reproduction(self):
        print(f"\nStarting selection and reproduction...")
        # Sort by fitness
        self.population.sort(key=lambda x: x['fitness'], reverse=True)
        elite_size = max(1, int(self.config['population_size'] * self.config['elite_percentage']))
        elite = self.population[:elite_size]
        print(f"Selecting {elite_size} elite individuals:")
        for i, individual in enumerate(elite):
            print(f"   Elite {i+1}: {individual['id']} (fitness: {individual['fitness']:.2f}%)")
        new_population = copy.deepcopy(elite)
        offspring_needed = self.config['population_size'] - len(new_population)
        print(f"Creating {offspring_needed} new individuals through crossover and mutation...")
        offspring_created = 0
        while len(new_population) < self.config['population_size']:
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            child1, child2 = crossover_genomes(parent1, parent2, self.config)
            child1 = mutate_genome(child1, self.config)
            if len(new_population) < self.config['population_size']:
                new_population.append(child1)
            child2 = mutate_genome(child2, self.config)
            if len(new_population) < self.config['population_size']:
                new_population.append(child2)
            offspring_created += 2
            if offspring_created % 4 == 0:
                print(f"   Created {min(offspring_created, offspring_needed)} of {offspring_needed} new individuals...")
        self.population = new_population[:self.config['population_size']]
        print(f"New generation created with {len(self.population)} individuals")
        print(f"   Elite preserved: {elite_size}")
        print(f"   New individuals: {len(self.population) - elite_size}")

    def tournament_selection(self, tournament_size: int = 3) -> dict:
        tournament = random.sample(self.population, min(tournament_size, len(self.population)))
        return max(tournament, key=lambda x: x['fitness'])

    def _update_adaptive_mutation(self):
        # Diversity measured via std of fitness in last generation
        if not self.generation_stats:
            self.config['current_mutation_rate'] = self.config['base_mutation_rate']
            return
        last_std = self.generation_stats[-1]['std_fitness']
        # Heuristic: more diversity -> lower mutation, low diversity -> higher
        # Normalize std roughly assuming fitness in [0,100]
        diversity_factor = min(1.0, last_std / 10.0)  # std 10% -> factor 1
        # Invert: low diversity (small std) should raise mutation
        inverted = 1 - diversity_factor
        new_rate = self.config['base_mutation_rate'] + (inverted - 0.5) * 0.4  # adjust +/-0.2 range
        new_rate = max(self.config['mutation_rate_min'], min(self.config['mutation_rate_max'], new_rate))
        self.config['current_mutation_rate'] = round(new_rate, 4)
        print(f"Adaptive mutation rate updated to {self.config['current_mutation_rate']} (std_fitness={last_std:.2f})")

    def check_convergence(self) -> bool:
        """
        Checks if the evolutionary process should terminate based on multiple criteria.
        
        This function implements comprehensive convergence checking to determine
        when the evolution should stop. It considers multiple termination conditions
        to balance optimization time with solution quality.
        
        Termination Criteria:
        1. Target fitness reached: Best individual achieves desired accuracy
        2. Maximum generations: Computational budget exhausted
        3. Stagnation detection: No improvement in recent generations
        
        Returns:
            bool: True if evolution should terminate, False to continue
        
        Design Philosophy:
        - Multiple criteria ensure robustness against different scenarios
        - Prevents infinite loops while allowing sufficient optimization time
        - Balances solution quality with computational efficiency
        """
        # CRITERION 1: TARGET FITNESS ACHIEVED
        # Stop if we've reached the desired accuracy threshold
        if self.best_individual and self.best_individual['fitness'] >= self.config['fitness_threshold']:
            print(f"Target fitness reached! ({self.best_individual['fitness']:.2f}% >= {self.config['fitness_threshold']}%)")
            return True
        
        # CRITERION 2: MAXIMUM GENERATIONS REACHED
        # Stop if we've exhausted our computational budget
        if self.generation >= self.config['max_generations']:
            print(f"Maximum generations reached ({self.generation}/{self.config['max_generations']})")
            return True
        
        # CRITERION 3: STAGNATION DETECTION
        # Stop if fitness hasn't improved significantly in recent generations
        if len(self.fitness_history) >= 3:
            recent = self.fitness_history[-3:]  # Last 3 generations
            fitness_range = max(recent) - min(recent)
            if fitness_range < 0.5:  # Less than 0.5% improvement
                print("Stagnation detected in last 3 generations")
                return True
        
        # CONTINUE EVOLUTION
        return False

    def evolve(self) -> dict:
        """
        Executes the complete hybrid neuroevolution process.
        
        This is the main entry point that orchestrates the entire evolutionary algorithm.
        It combines genetic algorithms with neural network training to evolve optimal
        architectures and hyperparameters.
        
        Evolution Process:
        1. Initialize random population of neural network architectures
        2. Evaluate fitness of each individual using threaded training
        3. Select elite individuals and create offspring through crossover/mutation
        4. Repeat until convergence criteria are met
        5. Return the best architecture found
        
        Returns:
            dict: Best genome found during evolution with highest fitness
        
        Key Features:
        - Adaptive mutation rates based on population diversity
        - Threaded fitness evaluation for performance (2x speedup)
        - Multiple convergence criteria for robust termination
        - Comprehensive logging and progress tracking
        """
        # EVOLUTION INITIALIZATION
        print("STARTING HYBRID NEUROEVOLUTION PROCESS (adaptive mutation)")
        print("="*60)
        print(f"Configuration:")
        print(f"   Population: {self.config['population_size']} individuals")
        print(f"   Maximum generations: {self.config['max_generations']}")
        print(f"   Target fitness: {self.config['fitness_threshold']}%")
        print(f"   Device: {device}")
        print("="*60)
        
        # CREATE INITIAL POPULATION
        # Generate diverse set of random architectures to start evolution
        self.initialize_population()
        
        # MAIN EVOLUTION LOOP
        # Continue until convergence criteria are satisfied
        while not self.check_convergence():
            # GENERATION HEADER
            print(f"\n{'='*80}")
            print(f"GENERATION {self.generation}")
            print(f"{'='*80}")
            
            # POPULATION EVALUATION (THREADED)
            # Evaluate fitness of all individuals using concurrent threads
            # This is the most computationally expensive part
            self.evaluate_population()
            
            # EARLY CONVERGENCE CHECK
            # Check if target fitness was reached during evaluation
            if self.check_convergence():
                break
            
            # ADAPTIVE PARAMETER ADJUSTMENT
            # Update mutation rate based on population diversity
            self._update_adaptive_mutation()
            
            # REPRODUCTION AND SELECTION
            # Create next generation through selection, crossover, and mutation
            self.selection_and_reproduction()
            
            # ADVANCE TO NEXT GENERATION
            self.generation += 1
            print(f"\nPreparing for next generation...")
        
        # EVOLUTION COMPLETION
        print(f"\n{'='*80}")
        print(f"EVOLUTION COMPLETED!")
        print(f"{'='*80}")
        print(f"Best individual found:")
        print(f"   ID: {self.best_individual['id']}")
        print(f"   Fitness: {self.best_individual['fitness']:.2f}%")
        print(f"   Origin generation: {self.generation}")
        print(f"   Total generations processed: {self.generation + 1}")
        
        return self.best_individual

print("HybridNeuroevolution class updated with adaptive mutation and interleaved early stopping")

## 7. Evolution Process Execution

In [ ]:
# ==========================================
# CONFIGURACIÓN DE DATASET - MODIFICAR AQUÍ
# ==========================================

# Para cambiar el dataset, modifica la línea correspondiente y ejecuta esta celda:

# Opción 1: Usar MNIST (28x28, grayscale, 10 classes)
# CONFIG['dataset'] = 'MNIST'

# Opción 2: Usar CIFAR-10 (32x32, RGB, 10 classes) - RECOMENDADO para mayor challenge
# CONFIG['dataset'] = 'CIFAR10'

# Opción 3: Usar dataset personalizado
# CONFIG['dataset'] = 'CUSTOM'
# CONFIG['dataset_path'] = r'E:\Neuroevolution\data\phd_data'  # Ajustar ruta según tu dataset

# ==========================================
# OTRAS CONFIGURACIONES OPCIONALES
# ==========================================


# Reconfigurar el dataset con los nuevos parámetros
CONFIG = configure_dataset(CONFIG, CONFIG['dataset'])

print("Current configuration:")
print(f"   Dataset: {CONFIG['dataset']}")
print(f"   Image size: {CONFIG['px_h']}x{CONFIG['px_w']}x{CONFIG['num_channels']}")
print(f"   Number of classes: {CONFIG['num_classes']}")
print(f"   Population: {CONFIG['population_size']} individuals")
print(f"   Maximum generations: {CONFIG['max_generations']}")
print(f"   Target fitness: {CONFIG['fitness_threshold']}%")
print(f"   Device: {device}")

# Recargar el dataset con la nueva configuración
print(f"\nReloading dataset with new configuration...")
train_loader, test_loader = load_dataset(CONFIG)

# Initialize neuroevolution system
start_time = datetime.now()
print(f"\nStarting neuroevolution at {start_time.strftime('%H:%M:%S')}")

# Create system instance
neuroevolution = HybridNeuroevolution(CONFIG, train_loader, test_loader)

# Execute evolution process
best_genome = neuroevolution.evolve()

end_time = datetime.now()
execution_time = end_time - start_time

print(f"\nProcess completed at {end_time.strftime('%H:%M:%S')}")
print(f"Total execution time: {execution_time}")
print(f"Total generations: {neuroevolution.generation}")
print(f"Best fitness achieved: {best_genome['fitness']:.2f}%")

## 8. Results Visualization and Analysis

In [ ]:
# Configure matplotlib style
plt.style.use('default')
sns.set_palette("husl")

# Function to visualize fitness evolution
def plot_fitness_evolution(neuroevolution):
    """Plots fitness evolution across generations."""
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics data to plot")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Extract data and filter 0.00 fitness
    generations = []
    avg_fitness = []
    max_fitness = []
    min_fitness = []
    std_fitness = []
    
    for stat in neuroevolution.generation_stats:
        # Only include if valid fitness (> 0.00)
        if stat['max_fitness'] > 0.00:
            generations.append(stat['generation'])
            avg_fitness.append(stat['avg_fitness'])
            max_fitness.append(stat['max_fitness'])
            min_fitness.append(stat['min_fitness'])
            std_fitness.append(stat['std_fitness'])
    
    if not generations:
        print("WARNING: No valid fitness data to plot (all are 0.00)")
        return
    
    # Graph 1: Fitness evolution
    ax1.plot(generations, max_fitness, 'g-', linewidth=2, marker='o', label='Maximum Fitness')
    ax1.plot(generations, avg_fitness, 'b-', linewidth=2, marker='s', label='Average Fitness')
    ax1.plot(generations, min_fitness, 'r-', linewidth=2, marker='^', label='Minimum Fitness')
    ax1.fill_between(generations, 
                     [max(0, avg - std) for avg, std in zip(avg_fitness, std_fitness)],
                     [avg + std for avg, std in zip(avg_fitness, std_fitness)],
                     alpha=0.2, color='blue')
    
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Fitness (%)')
    ax1.set_title('Fitness Evolution by Generation (Excluding 0.00%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add target fitness line
    ax1.axhline(y=CONFIG['fitness_threshold'], color='orange', linestyle='--', 
                label=f"Target ({CONFIG['fitness_threshold']}%)")
    ax1.legend()
    
    # Set Y axis limits for better visualization
    y_min = max(0, min(min_fitness) - 5)
    y_max = min(100, max(max_fitness) + 5)
    ax1.set_ylim(y_min, y_max)
    
    # Graph 2: Diversity (standard deviation)
    ax2.plot(generations, std_fitness, 'purple', linewidth=2, marker='D')
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Fitness Standard Deviation')
    ax2.set_title('Population Diversity (Excluding 0.00%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show additional information
    print(f"Plotted data:")
    print(f"   Generations with valid fitness: {len(generations)}")
    print(f"   Best fitness achieved: {max(max_fitness):.2f}%")
    print(f"   Final average fitness: {avg_fitness[-1]:.2f}%")
    if len(generations) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(generations)
        print(f"   WARNING: Excluded generations (0.00 fitness): {excluded}")

# Function to show detailed statistics
def show_evolution_statistics(neuroevolution):
    """Shows detailed evolution statistics."""
    print("DETAILED EVOLUTION STATISTICS")
    print("="*60)
    
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics available")
        return
    
    # Filter statistics with valid fitness
    valid_stats = [stat for stat in neuroevolution.generation_stats if stat['max_fitness'] > 0.00]
    
    if not valid_stats:
        print("WARNING: No valid statistics (all fitness are 0.00)")
        return
    
    final_stats = valid_stats[-1]
    
    print(f"Completed generations: {neuroevolution.generation}")
    print(f"Generations with valid fitness: {len(valid_stats)}")
    if len(valid_stats) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(valid_stats)
        print(f"WARNING: Generations with 0.00 fitness (excluded): {excluded}")
    
    print(f"\nFINAL STATISTICS (excluding 0.00 fitness):")
    print(f"   Final best fitness: {final_stats['max_fitness']:.2f}%")
    print(f"   Final average fitness: {final_stats['avg_fitness']:.2f}%")
    print(f"   Final minimum fitness: {final_stats['min_fitness']:.2f}%")
    print(f"   Final standard deviation: {final_stats['std_fitness']:.2f}%")
    
    # Progress across generations
    if len(valid_stats) > 1:
        initial_max = valid_stats[0]['max_fitness']
        final_max = valid_stats[-1]['max_fitness']
        improvement = final_max - initial_max
        
        print(f"\nPROGRESS:")
        print(f"   Initial fitness: {initial_max:.2f}%")
        print(f"   Final fitness: {final_max:.2f}%")
        print(f"   Total improvement: {improvement:.2f}%")
        if initial_max > 0:
            print(f"   Relative improvement: {(improvement/initial_max)*100:.1f}%")
    
    # Convergence analysis
    print(f"\nCONVERGENCE CRITERIA:")
    if neuroevolution.best_individual and neuroevolution.best_individual['fitness'] >= CONFIG['fitness_threshold']:
        print(f"   OK: Target fitness reached ({CONFIG['fitness_threshold']}%)")
    else:
        print(f"   ERROR: Target fitness NOT reached ({CONFIG['fitness_threshold']}%)")
    
    if neuroevolution.generation >= CONFIG['max_generations']:
        print(f"   TIME: Maximum generations reached ({CONFIG['max_generations']})")
    
    # Additional performance statistics
    all_max_fitness = [stat['max_fitness'] for stat in valid_stats]
    all_avg_fitness = [stat['avg_fitness'] for stat in valid_stats]
    
    print(f"\nGENERAL STATISTICS:")
    print(f"   Best fitness of entire evolution: {max(all_max_fitness):.2f}%")
    print(f"   Average fitness of entire evolution: {np.mean(all_avg_fitness):.2f}%")
    print(f"   Average improvement per generation: {(max(all_max_fitness) - min(all_max_fitness))/len(valid_stats):.2f}%")
    
    if neuroevolution.best_individual:
        print(f"\nBest individual ID: {neuroevolution.best_individual['id']}")
        print(f"Best individual fitness: {neuroevolution.best_individual['fitness']:.2f}%")

# Additional function for failure analysis
def analyze_failed_evaluations(neuroevolution):
    """Analyzes evaluations that resulted in 0.00 fitness."""
    print("\nFAILED EVALUATIONS ANALYSIS")
    print("="*50)
    
    total_generations = len(neuroevolution.generation_stats)
    failed_generations = len([stat for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00])
    
    if failed_generations == 0:
        print("OK: No failed evaluations (0.00 fitness)")
        return
    
    success_rate = ((total_generations - failed_generations) / total_generations) * 100
    
    print(f"Failure summary:")
    print(f"   Total generations: {total_generations}")
    print(f"   Failed generations: {failed_generations}")
    print(f"   Success rate: {success_rate:.1f}%")
    
    if failed_generations > 0:
        failed_gens = [stat['generation'] for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00]
        print(f"   Generations with failures: {failed_gens}")
        
        print(f"\nPossible causes of 0.00 fitness:")
        print(f"   • Errors in model architecture")
        print(f"   • Memory problems (GPU/RAM)")
        print(f"   • Invalid hyperparameter configurations")
        print(f"   • Errors during training")

# Execute visualizations
plot_fitness_evolution(neuroevolution)
show_evolution_statistics(neuroevolution)
analyze_failed_evaluations(neuroevolution)

## 9. BEST ARCHITECTURE FOUND

In [ ]:
def display_best_architecture(best_genome, config):
    """
    Shows the best architecture found in detailed and visual format.
    """
    print("="*60)
    print("        BEST EVOLVED ARCHITECTURE")
    print("="*60)
    
    # General information
    print(f"\nGENERAL INFORMATION:")
    print(f"   Genome ID: {best_genome['id']}")
    print(f"   Fitness Achieved: {best_genome['fitness']:.2f}%")
    print(f"   Generation: {neuroevolution.generation}")
    
    # Architecture details
    print(f"\nNETWORK ARCHITECTURE:")
    print(f"   Convolutional Layers: {best_genome['num_conv_layers']}")
    print(f"   Fully Connected Layers: {best_genome['num_fc_layers']}")
    
    print(f"\nCONVOLUTIONAL LAYER DETAILS:")
    for i in range(best_genome['num_conv_layers']):
        filters = best_genome['filters'][i]
        kernel = best_genome['kernel_sizes'][i]
        activation = best_genome['activations'][i % len(best_genome['activations'])]
        print(f"   Conv{i+1}: {filters} filters, kernel {kernel}x{kernel}, activation {activation}")
    
    print(f"\nFULLY CONNECTED LAYER DETAILS:")
    for i, nodes in enumerate(best_genome['fc_nodes']):
        print(f"   FC{i+1}: {nodes} neurons")
    print(f"   Output: {config['num_classes']} neurons (classes)")
    
    print(f"\nHYPERPARAMETERS:")
    print(f"   Optimizer: {best_genome['optimizer'].upper()}")
    print(f"   Learning Rate: {best_genome['learning_rate']:.4f}")
    print(f"   Dropout Rate: {best_genome['dropout_rate']:.3f}")
    print(f"   Activation Functions: {', '.join(best_genome['activations'])}")
    
    # Create and show final model
    print(f"\nCREATING FINAL MODEL...")
    try:
        final_model = EvolvableCNN(best_genome, config)
        total_params = sum(p.numel() for p in final_model.parameters())
        trainable_params = sum(p.numel() for p in final_model.parameters() if p.requires_grad)
        
        print(f"   Model created successfully")
        print(f"   Total parameters: {total_params:,}")
        print(f"   Trainable parameters: {trainable_params:,}")
        
        # Architecture summary
        print(f"\nCOMPACT SUMMARY:")
        print(f"   {final_model.get_architecture_summary()}")
        
    except Exception as e:
        print(f"   ERROR creating model: {e}")
    
    # Visualization in table format
    print(f"\nSUMMARY TABLE:")
    print(f"{'='*80}")
    print(f"{'Parameter':<25} {'Value':<30} {'Description':<25}")
    print(f"{'='*80}")
    print(f"{'ID':<25} {best_genome['id']:<30} {'Unique identifier':<25}")
    print(f"{'Fitness':<25} {best_genome['fitness']:.2f}%{'':<25} {'Accuracy achieved':<25}")
    print(f"{'Conv Layers':<25} {best_genome['num_conv_layers']:<30} {'Convolutional layers':<25}")
    print(f"{'FC Layers':<25} {best_genome['num_fc_layers']:<30} {'FC layers':<25}")
    print(f"{'Optimizer':<25} {best_genome['optimizer']:<30} {'Optimization algorithm':<25}")
    print(f"{'Learning Rate':<25} {best_genome['learning_rate']:<30} {'Learning rate':<25}")
    print(f"{'Dropout':<25} {best_genome['dropout_rate']:<30} {'Dropout rate':<25}")
    print(f"{'='*80}")
    
    # Comparison with initial configuration
    print(f"\nCOMPARISON WITH OBJECTIVES:")
    if best_genome['fitness'] >= config['fitness_threshold']:
        print(f"   TARGET: OK Fitness objective REACHED ({best_genome['fitness']:.2f}% >= {config['fitness_threshold']}%)")
    else:
        print(f"   TARGET: ERROR Fitness objective NOT reached ({best_genome['fitness']:.2f}% < {config['fitness_threshold']}%)")
    
    print(f"   TIME: Generations used: {neuroevolution.generation}/{config['max_generations']}")
    
    # Save information to JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"best_architecture_{timestamp}.json"
    
    results_data = {
        'timestamp': timestamp,
        'execution_time': str(execution_time),
        'config_used': config,
        'best_genome': best_genome,
        'final_generation': neuroevolution.generation,
        'evolution_stats': neuroevolution.generation_stats
    }
    
    try:
        with open(results_file, 'w') as f:
            json.dump(results_data, f, indent=2, default=str)
        print(f"\nResults saved to: {results_file}")
    except Exception as e:
        print(f"\nWARNING: Error saving results: {e}")
    
    print(f"\nHYBRID NEUROEVOLUTION COMPLETED SUCCESSFULLY!")
    print(f"{'='*60}")

# Show the best architecture found
display_best_architecture(best_genome, CONFIG)

In [ ]:
## 📚 Code Documentation Summary

This notebook now includes comprehensive English comments throughout all major functions and classes. Here's what has been documented:

### 🧵 **Threading Implementation** 
- **`_thread_worker()`**: Core threading logic with detailed synchronization explanations
- **`evaluate_population()`**: Producer-consumer pattern and thread coordination 
- **Thread Safety**: Lock usage, atomic operations, and race condition prevention

### 🔬 **Fitness Evaluation System**
- **`evaluate_fitness()`**: Complete neural network training and evaluation pipeline
- **`_train_one_epoch()`**: Optimized training with batch limiting for speed
- **`_evaluate()`**: Fast model evaluation with accuracy calculation

### 🧬 **Genetic Algorithm Core**
- **`evolve()`**: Main evolution orchestration and flow control
- **`initialize_population()`**: Random genome generation for diversity
- **`check_convergence()`**: Multi-criteria termination detection

### 🏗️ **Neural Architecture**
- **`EvolvableCNN`**: Dynamic CNN construction from genetic specifications
- **`_build_conv_layers()`**: Feature extraction layer assembly
- **Architecture flexibility**: Variable layers, filters, and hyperparameters

### 🔄 **Process Flow Documentation**
1. **Initialization**: Random population generation with diverse architectures
2. **Evaluation**: Concurrent fitness assessment using 2 worker threads  
3. **Selection**: Elite preservation and tournament selection
4. **Reproduction**: Crossover and adaptive mutation for next generation
5. **Convergence**: Multiple termination criteria for robust stopping

### 🛡️ **Thread Safety Features**
- **Locks**: `genome_lock` for data access, `results_lock` for logging
- **Atomic Operations**: Thread-safe list operations and state updates
- **Synchronization**: Proper thread joining and cleanup procedures

### ⚡ **Performance Optimizations**
- **Concurrent Processing**: 2x speedup through parallel genome evaluation
- **Batch Limiting**: Faster training through reduced batch processing
- **Early Stopping**: Prevents overfitting and reduces computation time
- **Memory Efficiency**: Proper GPU memory management and cleanup

All critical functions now have detailed docstrings explaining purpose, parameters, return values, algorithms, and design decisions. The threading implementation is thoroughly documented with explanations of synchronization mechanisms and race condition prevention.